# __Delsys FileReaderAPI - C# .NET Overview__
This document outlines how to directly read SHPF files within a .NET Interactive environment using Delsys' FileReaderAPI

## __Requirements__
1) .NET 6 SDK https://dotnet.microsoft.com/download/dotnet/6.0
2) .NET Interactive Notebooks https://marketplace.visualstudio.com/items?itemName=ms-dotnettools.dotnet-interactive-vscode


## Load FileReader DLL from Trigno Discover's Program files
Loads the dll file and Plotly.NET needed to perform the reading and plotting/graphing task.

In [ ]:
#r "FileReader.dll"
#r "nuget: Plotly.NET, 2.0.0-preview.16"

## Setup the FileReader

First import FileReader, then initalize the FileReadingStream by passing the shpf file location. Then initalize the ReadFile object by passing it the FileReadingStream object.

In [ ]:
using FileReader;

string shpffile = "Forearm_Pronation_Supination_1.shpf";
FileReadingStream fileReadingStream = new FileReadingStream(shpffile);
ReadFile file = new ReadFile(fileReadingStream);


## Retrieving Data
After the file is parsed all of the individual sensor component objects are stored in a list. The file provided with this example only has one sensor's data, therefore we will reference index 0 to retrieve the first component in that list. If more sensors were used during a collection, you may loop through the sensor objects by using the total number of sensors in the file. To see all of the metadata and methods associated with each component object see IFileComponent Interface located at FileReader.Interface

In [ ]:
using FileReader.Interface;

List<IFileComponent> components = file.ParsedFile.Components;

IFileComponent component1 = components[0];
System.Console.WriteLine("Component Name: " + component1.Name);

## Get data from a single channel 
Each component object contains a list of channel objects. This example will pull the first channel from component1 object. Each channel object has a variety of metadata along with all of the channel data. To see all of the metadata and methods associated with each channel object see IFileChannel Interface located at FileReader.Interface

In [ ]:
IFileChannel channel1 = component1.Channels[0];
System.Console.WriteLine("Channel Name: " + channel1.Name);

## Get all data from a component
You can get all channel data (for a single sensor component) and other metadata with these method calls.

In [ ]:
// Gets all of the data associated with current component (i.e. all data from every channel belonging to the component).
List<List<double>> data = new List<List<double>>();
foreach(var channel in component1.Channels){
    List<double> dataTemp = new List<double>();
    foreach(var dataList in channel.Data){
        foreach(var sample in dataList.Samples){
            dataTemp.Add(sample);
        }
    }
    data.Add(dataTemp);
}

// Gets channel names from all channels belonging to current component.
List<string> names = new List<string>();
foreach(var channel in component1.Channels){
    names.Add(channel.Name);
}
// Gets sample rates of every channel belonging to current component.
List<double> sampleRates = new List<double>();
foreach(var channel in component1.Channels){
    sampleRates.Add(channel.SampleRate);
}

// Gets all units from every channel belonging to current component.
List<string> units = new List<string>();
foreach(var channel in component1.Channels){
    units.Add(channel.Units.ToString());
}

System.Console.WriteLine("Component Channel Data:");
for (int i=0; i<data.Count(); i++){
    System.Console.WriteLine("--Channel: " + names[i] + 
    " Sample Rate: " + Math.Round(sampleRates[i], 4) + 
    " Unit: " + units[i] + 
    " Data Length: " + data[i].Count());
}


## Plotting single channel of data
Here the EMG data is plotted on its own figure

In [ ]:
using Plotly.NET;
using Plotly.NET.LayoutObjects;

// Calculates the time axis for first channel based on channel sample rate
List<double> channel_time = new List<double>();
for(double i = 0; i< data[0].Count(); i+=1){
    channel_time.Add(i/sampleRates[0]);
}

// Set x-axis
LinearAxis xAxis = new LinearAxis();
xAxis.SetValue("title", "Time (s)");

// Set y-axis
LinearAxis yAxis = new LinearAxis();
yAxis.SetValue("title", units[0]);

// Create plot layout
Layout layout = new Layout();
layout.SetValue("xaxis", xAxis);
layout.SetValue("yaxis", yAxis);
layout.SetValue("title", "EMG Data");

// Set plot data
Trace trace = new Trace("scatter");
trace.SetValue("x", channel_time);
trace.SetValue("y", data[0]);
trace.SetValue("mode", "lines");

// Show plot
GenericChart
    .ofTraceObject(true, trace)
    .WithLayout(layout)
    .Show();

## Plot all data from sensor component
Here all of the data is plotted based on the channel type (EMG, ACC, GYRO)

In [ ]:
List<string> channel_types = new List<string>();
for (int i=0;i<component1.Channels.Count(); i++){
    if (channel_types.Contains(component1.Channels[i].ChannelType.ToString())){
        continue;
    }
    else{
        channel_types.Add(component1.Channels[i].ChannelType.ToString());
    }
}

List<List<GenericChart.GenericChart>> plots = new List<List<GenericChart.GenericChart>>();

//Loop all unique channel types
for (int i=0; i<channel_types.Count(); i++)
{
    List<GenericChart.GenericChart> uniqueTypePlot = new List<GenericChart.GenericChart>();

    LinearAxis xAxis = new LinearAxis();
    xAxis.SetValue("title", "Time (s)");
    
    //Loop all channel data - if channel type matches, add the channel data to that plot
    for (int k=0; k<data.Count(); k++)
    {
        if (component1.Channels[k].ChannelType.ToString() == channel_types[i])
        {
            //x-axes time values based on sampling rates
            List<double> time = new List<double>();
            for (double j = 0; j< data[i].Count(); j++)
            {
                time.Add(j/sampleRates[k]);
            }

            Trace trace = new Trace("scatter");
            trace.SetValue("x", time);
            trace.SetValue("y", data[k]);
            trace.SetValue("mode", "lines");
            
            trace.SetValue("name", names[k]);

            LinearAxis yAxis = new LinearAxis();
            yAxis.SetValue("title", units[k]);

            Layout layout = new Layout();
            layout.SetValue("xaxis", xAxis);
            layout.SetValue("yaxis", yAxis);
            layout.SetValue("showlegend", true);
            layout.SetValue("title", channel_types[i]);

            var plotPart = GenericChart.ofTraceObject(true, trace).WithLayout(layout);
            uniqueTypePlot.Add(plotPart);
        }
    }
    plots.Add(uniqueTypePlot);
}

for (int i = 0; i<plots.Count(); i++)
{
    var plot = Plotly.NET.Chart.Combine(plots[i]);
    plot.Show();
}